# SwissHouseRSlaTank notebook example

In this example, the usage of the model "SwissHouseRad-v0" is demonstrated.

First, we import Energym and create the simulation environment by specifying the model, a weather file and the number of simulation days.

In [3]:
import sys
sys.path.insert(0, "/home/lesong/energym/")
import energym

weather = "CH_BS_Basel"
env = energym.make("SwissHouseRSlaTank-v0", weather=weather, simulation_days=20)

the initial variables are {'uHP': 0, 'uRSla': 0}


The control inputs can be inspected using the `get_inputs_names()` method.

In [4]:
inputs = env.get_inputs_names()
outputs = env.get_outputs_names()
print("inputs:", inputs)
print("outputs:", outputs)

inputs: ['uHP', 'uRSla']
outputs: ['TOut.T', 'heaPum.COP', 'heaPum.COPCar', 'heaPum.P', 'heaPum.QCon_flow', 'heaPum.QEva_flow', 'heaPum.TConAct', 'heaPum.TEvaAct', 'preHea.Q_flow', 'sla.QTot', 'sla.heatPortEmb[1].T', 'sla.m_flow', 'tanSH.heaPorSid.T', 'temRoo.T', 'weaBus.HDifHor', 'weaBus.HDirNor', 'weaBus.HGloHor', 'weaBus.HHorIR']


In [5]:
[ p for p in env.vrs if "tem" in p]

['tanSH.indTanHex.hAPipIns[1].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[2].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[3].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[4].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[5].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[6].temperatureDependent',
 'tanSH.indTanHex.hA_temperatureDependent',
 'tanSH.indTanHex.temSenHex[1].T',
 'tanSH.indTanHex.temSenHex[1].port.Q_flow',
 'tanSH.indTanHex.temSenHex[1].port.T',
 'tanSH.indTanHex.temSenHex[2].T',
 'tanSH.indTanHex.temSenHex[2].port.Q_flow',
 'tanSH.indTanHex.temSenHex[2].port.T',
 'tanSH.indTanHex.temSenHex[3].T',
 'tanSH.indTanHex.temSenHex[3].port.Q_flow',
 'tanSH.indTanHex.temSenHex[3].port.T',
 'tanSH.indTanHex.temSenHex[4].T',
 'tanSH.indTanHex.temSenHex[4].port.Q_flow',
 'tanSH.indTanHex.temSenHex[4].port.T',
 'tanSH.indTanHex.temSenHex[5].T',
 'tanSH.indTanHex.temSenHex[5].port.Q_flow',
 'tanSH.indTanHex.temSenHex[5].port.T',
 'tanSH.indTanHex.temSenHex[6].T',
 't

To run the simulation, a number of steps is specified (here 288 steps per day for 10 days), a control input is specified and passed to the simulation model with the `step()` method. To generate some plots later on, we save all the outputs in lists.

In [7]:
from scipy import signal
import math

steps = 288*5
out_list = []
outputs = env.get_output()
controls = []
hour = 0
for i in range(steps):
    control = {}
    control['uHP'] = [0.5*(signal.square(0.1*i)+1.0)]
    control['uRSla'] = [0.5*(math.sin(0.01*i)+1.0)]
    controls +=[ {p:control[p][0] for p in control} ]
    outputs = env.step(control)
    _,hour,_,_ = env.get_date()
    out_list.append(outputs)

[OK] <JMIRuntime><value name="build_date">"Feb 24 2021"</value> <value name="build_time">"21:25:41"</value></JMIRuntime>
[WARNING] <ModelicaMessage category="warning"><value name="msg">"... loading ""tab1"" from ""/home/lesong/Energym_runs/1669962761_1294545/resources/0/weather.mos""&##10;"</value></ModelicaMessage>
[WARNING] <ModelicaMessage category="warning"><value name="msg">"... loading ""tab1"" from ""/home/lesong/Energym_runs/1669962761_1294545/resources/0/weather.mos""&##10;"</value></ModelicaMessage>
[ERROR] <DoStep category="error">Cannot take a step past the <value name="stop_time">        1.7280000000000000E+06</value>. Asked <value name="final_time">        1.7283000000000000E+06</value>.</DoStep>


FMICallException: fmi2DoStep failed with status 3 (error).

Since the outputs are given as dictionaries and are collected in lists, we can simply load them as a pandas.DataFrame.

In [ ]:
import pandas as pd
out_df = pd.DataFrame(out_list)

: 

In [ ]:
out_df

: 

To generate plots, we can directly get the data from the DataFrames, by using the key names. Displayed are the room temperature, the supply temperature and the return temperature, as well as the external temperature, and the heat pump energy.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

f, (ax1,ax2,ax3) = plt.subplots(3,figsize=(10,15))#


ax1.plot(out_df['temRoo.T']-273.15, 'r')
ax1.plot(out_df['sla.heatPortEmb[1].T']-273.15, 'b--')
ax1.plot(out_df['heaPum.TEvaAct']-273.15, 'orange')
ax1.set_ylabel('Temp')
ax1.set_xlabel('Steps')

ax2.plot(out_df['TOut.T']-273.15, 'r')
ax2.set_ylabel('Temp')
ax2.set_xlabel('Steps')

ax3.plot(out_df['heaPum.QCon_flow'], 'g')
ax3.set_ylabel('Energy')
ax3.set_xlabel('Steps')

plt.subplots_adjust(hspace=0.4)

plt.show()

: 

To end the simulation, the `close()` method is called. It deletes files that were produced during the simulation and stores some information about the simulation in the *energym_runs* folder.

In [ ]:
env.close()

: 

: 